In [1]:
import re
import xml.etree.ElementTree as ET

In [2]:
DESC = set('⿰	⿱	⿲	⿳	⿴	⿵	⿶	⿷	⿸	⿹	⿺	⿻'.split())

In [3]:
RE_CJK = re.compile(r'[\u4e00-\u9fff]+')
RE_braces = re.compile(r'{[^}]*}|[^{}]+')

In [4]:
cjk_ranges = [
        ( 0x4E00,  0x62FF),
        ( 0x6300,  0x77FF),
        ( 0x7800,  0x8CFF),
        ( 0x8D00,  0x9FCC),
        ( 0x3400,  0x4DB5),
        (0x20000, 0x215FF),
        (0x21600, 0x230FF),
        (0x23100, 0x245FF),
        (0x24600, 0x260FF),
        (0x26100, 0x275FF),
        (0x27600, 0x290FF),
        (0x29100, 0x2A6DF),
        (0x2A700, 0x2B734),
        (0x2B740, 0x2B81D),
        (0x2B820, 0x2CEAF),
        (0x2CEB0, 0x2EBEF),
        (0x2F800, 0x2FA1F)
    ]

def is_cjk(char):
    char = ord(char)
    for bottom, top in cjk_ranges:
        if char >= bottom and char <= top:
            return True
    return False

In [5]:
ct = 1
ct_ = 0

curr_iter = 1
MAX_ITER = 2500000

bad_chars = []
char_to_desc = {}

text = None

context = ET.iterparse('enwiktionary-20220101-pages-meta-current.xml')
event, root = next(context)

for _, elem in context:
    if curr_iter > MAX_ITER:
        break
    curr_iter += 1
#     print(f'\riter={curr_iter}', end='')
    
    if 'page' in elem.tag:
        
        title = [x for x in elem.getchildren() if 'title' in x.tag][0].text
        
        if title and len(title) == 1 and is_cjk(title):
            character = title
            rev = elem.find('{http://www.mediawiki.org/xml/export-0.10/}revision')
            if rev:
                text = rev.find('{http://www.mediawiki.org/xml/export-0.10/}text').text
                text = text.replace('\n', '')
                ids = [line for line in re.findall(RE_braces, text) if (set(line) & DESC)]
                
                if ids:
                    char_to_desc[character] = ids
                    print(f'\r{ct} success, {ct_} failures; iter={curr_iter}', end='')
                    ct += 1
                    
                else:
                    bad_chars.append(character)
                    ct_ += 1
                    root.clear()
                    continue
            
            else:
                bad_chars.append(character)
                ct_ += 1
                root.clear()
                continue
    
    root.clear()

21203 success, 97 failures; iter=2496155

In [10]:
import json

with open('raw_01.json', 'w') as f:
    json.dump(char_to_desc, f)

with open('bad_01.json', 'w') as f:
    json.dump(bad_chars, f)